In [6]:
# Install necessary libraries
!pip install datasets

# Importing required libraries for dataset, model, tokenizer, training, and evaluation
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from IPython import get_ipython
from IPython.display import display

In [7]:
# 1. Load Dataset
# Load the Bengali transliteration dataset
raw_dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Split dataset into training and validation sets (90% training, 10% validation)
train_data = raw_dataset['train'].train_test_split(test_size=0.1, seed=42)['train'] # Added seed for reproducibility
val_data = raw_dataset['train'].train_test_split(test_size=0.1, seed=42)['test'] # Added seed for reproducibility

# 2. Preprocessing
# Define model name for T5 and load its tokenizer
model_name = "google/mt5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Tokenize function for preprocessing the data
def preprocess_data(examples):
    # Tokenize input and target sequences with padding and truncation to fixed length
    # Access the correct columns based on the dataset structure
    inputs = tokenizer(examples['bn'], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples['rm'], padding="max_length", truncation=True, max_length=128)

    # Assign the tokenized target as labels for the model
    inputs['labels'] = targets['input_ids']
    return inputs

# Apply the preprocessing function to the training and validation datasets
train_dataset = train_data.map(preprocess_data, batched=True)
val_dataset = val_data.map(preprocess_data, batched=True)

# 3. Model Selection
# Load the T5 model for conditional generation
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 4. Training
# Define training arguments like learning rate, batch size, number of epochs, etc.
training_args = TrainingArguments(
    output_dir="./results", # Directory to store the results
    evaluation_strategy="epoch", # Evaluate model after each epoch
    learning_rate=5e-5, # Learning rate
    per_device_train_batch_size=16, # Batch size for training
    per_device_eval_batch_size=16, # Batch size for evaluation
    num_train_epochs=5, # Number of training epochs
    weight_decay=0.01, # Weight decay for regularization
    save_steps=1000, # Save model every 1000 steps
    save_total_limit=2, # Keep only 2 most recent checkpoints
    logging_dir="./logs", # Directory to store logs
    logging_steps=500, # Log every 500 steps
)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/4505 [00:00<?, ? examples/s]

Map:   0%|          | 0/501 [00:00<?, ? examples/s]

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
# Initialize the Trainer with the model, arguments, and datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start the training process
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: khalidbinselim (khalidbinselim-shahjalal-university-of-science-and-techn). Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


KeyboardInterrupt: 